# Data Cleaning

In [1]:
import nltk
import pandas as pd
import re, string
import os
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from nltk.probability import FreqDist

#data loading
data = pd.ExcelFile('C:/Users/utsav/OneDrive/UIC/Fall_2023/CS_583/Project/training-Obama-Romney-tweets.xlsx')
obama = pd.read_excel(data, 'Obama')
#data cleaning
obama = obama[1:]
obama = obama.drop(['Unnamed: 0', 'date', 'time', 'Unnamed: 5'], axis=1)
obama = obama.rename(columns={'Unnamed: 4': 'class', 'Anootated tweet': 'tweet'})

In [2]:
print(obama['class'].value_counts())

-1            1922
0             1896
1             1653
2             1474
0               82
2               70
-1              46
1               26
irrevelant      23
irrelevant       1
Name: class, dtype: int64


In [3]:
obama = obama.dropna()
obama = obama[obama['class'].isin(['-1', '0', '1',-1,0,1])]

In [4]:
print(obama['class'].value_counts())

-1    1922
0     1895
1     1653
0       82
-1      46
1       26
Name: class, dtype: int64


In [5]:
obama['sentiment'] = obama['class'].apply(lambda x: 'positive' if x == 1 else 'neutral' if x==0 else 'negative')

In [6]:
def clean(text):
    text = text.lower()
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)
    text = re.sub(r'www.[^ ]+', '', text)
    text = re.sub(r'[^a-z]', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text

regexp = RegexpTokenizer('\w+')
import spacy
nltk.download('stopwords')
def tokenize(text):
    
    stop_words = stopwords.words('english')
    text = clean(text)
    #text = nltk.word_tokenize(text)
    text = regexp.tokenize(text)
    text = [w for w in text if w not in stop_words]
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utsav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
obama['tweet_token'] = obama['tweet'].apply(lambda stext: tokenize(str(stext)))

#remove words with length less than 2
obama['tweet_string'] = obama['tweet_token'].apply(lambda x:' '.join([item for item in x if len(item)>2]))
all_words = ' '.join([text for text in obama['tweet_string']])
tokenized_obama = nltk.tokenize.word_tokenize(all_words)
fdist = FreqDist(tokenized_obama)
obama['tweet_string_fdist'] = obama['tweet_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 4 ]))
#fdist
#lemmatize
wordnet_lem = WordNetLemmatizer()
obama['tweet'] = obama['tweet_string_fdist'].apply(wordnet_lem.lemmatize)
# remove tweets with 1 word
#obama = obama[obama['tweet'].apply(lambda x: len(x.split())>1)]
obama.dropna(inplace=True)
obama = obama.drop(['tweet_token', 'tweet_string', 'tweet_string_fdist'], axis=1)
print(obama.shape)
obama.head(5)


(5624, 3)


,tweet,class,sentiment
1,wore barack obama look jason journal,0,neutral
3,obama debates ass tonight,1,positive
5,point afraid understand bigger picture dont ca...,0,neutral
7,raised democrat left party years ago never see...,-1,negative
8,obama camp afford lower expectations tonight d...,0,neutral


In [8]:
print(obama['sentiment'].value_counts())

negative    2076
neutral     1895
positive    1653
Name: sentiment, dtype: int64


In [9]:
obama = obama.dropna()

In [10]:
obama.to_csv('C:/Users/utsav/OneDrive/UIC/Fall_2023/CS_583/Project/Cleaned/obama_cleaned.csv', index=False)